# `분석 목적` 제품 수익성 평가를 위한 기본 매출 지표 분석

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

### 데이터 로드

In [25]:
df = pd.read_csv(r'data-files\06-product_mart.csv')
df.head()

,order_id,quantity,price,discount_ratio,cost,category,product_id,order_date
0,1.000,23.000,37,0.131,32,e,17.000,2023-02-07
1,150.000,23.000,37,0.072,32,e,17.000,2023-01-18
2,811.000,23.000,37,0.110,32,e,17.000,2023-02-22
3,1518.000,23.000,37,0.090,32,e,17.000,2023-02-05
4,1632.000,23.000,37,0.140,32,e,17.000,2023-01-31


In [26]:
# 데이터 파악
pd.set_option('display.float_format', '{:.3f}'.format)
display(df.describe())
display(df.info())

,order_id,quantity,price,discount_ratio,cost,product_id
count,5000.000,5000.000,5000.000,5000.000,5000.000,5000.000
mean,2500.500,6.068,96.417,0.014,70.400,10.477
std,1443.520,5.405,110.902,0.022,94.494,5.775
min,1.000,1.000,27.000,0.000,11.000,1.000
25%,1250.750,3.000,44.000,0.002,23.000,6.000
50%,2500.500,4.000,55.000,0.007,35.000,10.000
75%,3750.250,8.000,63.000,0.018,40.000,16.000
max,5000.000,46.000,365.000,0.303,296.000,20.000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_id        5000 non-null   float64
 1   quantity        5000 non-null   float64
 2   price           5000 non-null   int64  
 3   discount_ratio  5000 non-null   float64
 4   cost            5000 non-null   int64  
 5   category        5000 non-null   object 
 6   product_id      5000 non-null   float64
 7   order_date      5000 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 312.6+ KB


None

In [27]:
# order_date 자료형 변환 (object -> datetime)
df['order_date'] = pd.to_datetime(df['order_date'])

# product_id 자료형 변환 (float -> objdect)
df['product_id'] = df['product_id'].astype(int).astype(str)

In [28]:
# 범주형 데이터 관계 파악
df.groupby(['category', 'product_id']).size()

category  product_id
a         10            224
          13            248
          2             226
          6             269
          7             249
          9             277
b         18            241
          5             257
c         14            205
d         11            259
          16            256
          3             239
e         1             273
          17            252
          20            256
          4             247
          8             255
f         15            261
g         12            253
          19            253
dtype: int64

### 매출 지표 계산

(1) 계산 및 컬럼 추가

In [29]:
df.head()

,order_id,quantity,price,discount_ratio,cost,category,product_id,order_date
0,1.000,23.000,37,0.131,32,e,17,2023-02-07
1,150.000,23.000,37,0.072,32,e,17,2023-01-18
2,811.000,23.000,37,0.110,32,e,17,2023-02-22
3,1518.000,23.000,37,0.090,32,e,17,2023-02-05
4,1632.000,23.000,37,0.140,32,e,17,2023-01-31


In [30]:
# 순가격 (할인율 적용가)
df['net_price'] = df['price'] * (1-df['discount_ratio'])

In [31]:
# 총 순매출
df['total_net_revenue'] = df['net_price'] * df['quantity']

In [32]:
# 할인율 무시 총 매출
df['total_revenue'] = df['price'] * df['quantity']

In [33]:
# 총 비용
df['total_cost'] = df['cost'] * df['quantity']

In [34]:
# 단위 할인금
df['discount'] = df['price'] * df['discount_ratio']

In [35]:
# 총 할인금
df['total_discount'] = df['discount'] * df['quantity']

In [45]:
# 제품별 기준 매출 데이터 집계
df_agg = df.groupby(['product_id', 'category', 'price'])[[
    'quantity'
    , 'total_net_revenue'
    , 'total_revenue'
    , 'total_cost'
    , 'total_discount'
]].sum().reset_index()

df_agg.head()

,product_id,category,price,quantity,total_net_revenue,total_revenue,total_cost,total_discount
0,1,e,38,3423.000,125043.635,130074.000,78729.000,5030.365
1,10,a,27,874.000,23359.792,23598.000,9614.000,238.208
2,11,d,63,4410.000,261461.998,277830.000,180810.000,16368.002
3,12,g,365,649.000,235316.180,236885.000,192104.000,1568.820
4,13,a,49,969.000,47037.010,47481.000,35853.000,443.990


In [50]:
# 총 매출 대비 순이익 비율
df_agg['net_profit_ratio'] = (df_agg['total_revenue'] - df_agg['total_cost'] - df_agg['total_discount']) / df_agg['total_revenue']

df_agg.head()

,product_id,category,price,quantity,total_net_revenue,total_revenue,total_cost,total_discount,net_profit_ratio
0,1,e,38,3423.000,125043.635,130074.000,78729.000,5030.365,0.356
1,10,a,27,874.000,23359.792,23598.000,9614.000,238.208,0.582
2,11,d,63,4410.000,261461.998,277830.000,180810.000,16368.002,0.290
3,12,g,365,649.000,235316.180,236885.000,192104.000,1568.820,0.182
4,13,a,49,969.000,47037.010,47481.000,35853.000,443.990,0.236


(2) 계산 검증

In [52]:
# 발생비용 비율
df_agg['cost_ratio'] = df_agg['total_cost'] / df_agg['total_revenue']

In [55]:
# 할인율
df_agg['discount_ratio'] = df_agg['total_discount'] / df_agg['total_revenue']

In [58]:
# 순이익 비율 + 발생비용 비율 + 할인 비율 = 1 검증
df_agg['net_profit_ratio'] + df_agg['cost_ratio'] + df_agg['discount_ratio']

0    1.000
1    1.000
2    1.000
3    1.000
4    1.000
5    1.000
6    1.000
7    1.000
8    1.000
9    1.000
10   1.000
11   1.000
12   1.000
13   1.000
14   1.000
15   1.000
16   1.000
17   1.000
18   1.000
19   1.000
dtype: float64

In [51]:
# 순수익금
df_agg['total_net_profit'] = df_agg['total_revenue'] - df_agg['total_cost'] - df_agg['total_discount']

In [60]:
# 전체 매출 - 발생비용 - 할인액 - 순이익금 = 0 검증
df_agg['total_revenue'] - df_agg['total_cost'] - df_agg['total_discount'] - df_agg['total_net_profit']

0    0.000
1    0.000
2    0.000
3    0.000
4    0.000
5    0.000
6    0.000
7    0.000
8    0.000
9    0.000
10   0.000
11   0.000
12   0.000
13   0.000
14   0.000
15   0.000
16   0.000
17   0.000
18   0.000
19   0.000
dtype: float64

### 시각화

In [62]:
fig = px.scatter(
    df_agg
    , title = 'Product Portfolio Analysis'
    , x = 'total_revenue'
    , y = 'net_profit_ratio'
    , hover_name = 'product_id'
    , size = 'total_net_profit'
    , color = 'discount_ratio'
)

fig.show()